In [2]:
!pip install geopandas
!pip install osmnx
!pip install scipy
!pip installn networkx

^C


ERROR: unknown command "installn" - maybe you meant "install"



In [5]:
import pickle

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

## Istruzione

Utilizzeremo i seguenti dati:
1. **Biblioteche**
2. **Scuole**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

In [6]:
# Carichiamo Graph Ciclabili
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "rb") as f:
    G_strade_ciclabili = pickle.load(f)

In [7]:
G_compose = G_strade_ciclabili.copy()

### Aggiunta di **Biblioteche** come **"poi"**

1. Carichiamo il file dei biblioteche da "clean"
2. Utilizziamo **"connect_poi_nodes_to_graph()"** per connettere il geodataframe come "poi" al grafo delle strade/ciclabili.

In [9]:
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_biblioteche,
                                                                     tipo = "biblioteche",
                                                                     poi = True,
                                                                     artificial=True)

### Aggiunta di **Scuole** come **"poi"**

In [10]:
gdf_scuole = gpd.read_file(PATH_SCUOLE_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_biblioteche,
                                                                     tipo = "scuole",
                                                                     poi = True,
                                                                     artificial=True)

## Utilizziamo algoritmo di **Steiner Tree**

Una volta aggiunti i "poi" al grafo delle strade/ciclabili, eseguiamo l'algoritmo che troverà i percorsi minimi per connettere tutti i "poi" tra di loro con il minor "peso" possibile.  
I pesi sono stati già accuratamente inseriti sugli archi delle strade, mentre quelli delle ciclabili e degli archi all'interno dei poligoni dei "poi" sono = 0, in modo da preferire sempre quei percorsi quando possibile (sfruttare ciclabili pre-esistenti il più possibile).

In [11]:
poi_nodes = [n for n, d in G_compose.nodes(data=True) if d.get("poi") == True]
G_sport_tempo_libero = nx.MultiDiGraph(steiner_tree(G_compose, terminal_nodes=poi_nodes, weight='weight')).to_undirected()

Risultato lo salviamo in Clean di Analisi (per ora anche il pickle in staging) in geojson pronto da visualizzare

In [12]:
with open("..\Data\Staging\Analisi\Istruzione_clean.pickle", "wb") as f:
    pickle.dump(G_sport_tempo_libero, f)

gdf_steiner = ox.graph_to_gdfs(G_sport_tempo_libero, edges=True, nodes=False)
gdf_steiner.to_file("..\Data\Clean\Analisi\Istruzione_clean.geojson", driver="GeoJSON")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\apera\AppData\Local\Temp\ipykernel_26128\1417971601.py:1: SyntaxWarning: invalid escape sequence '\D'
  with open("..\Data\Staging\Analisi\Istruzione_clean.pickle", "wb") as f:
C:\Users\apera\AppData\Local\Temp\ipykernel_26128\1417971601.py:5: SyntaxWarning: invalid escape sequence '\D'
  gdf_steiner.to_file("..\Data\Clean\Analisi\Istruzione_clean.geojson", driver="GeoJSON")
